In [ ]:
from coincidence_matching import match_parts
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch

from brepmatching.data import BRepMatchingDataset

In [ ]:
ds = BRepMatchingDataset('../data/GeoFullSet.zip','../data/GeoFullSet2.pt', test_size=0, val_size=0)

In [ ]:
data = torch.load('../data/GeoFullSetCacheSplitNoNaN.pt')

In [ ]:
data.keys()

In [ ]:
len(data['original_index'])

In [ ]:
len(data['preprocessed_data'])

In [ ]:
d = data['preprocessed_data'][0]

In [ ]:
d.bl_exact_vertices_matches

In [ ]:
with ZipFile('../data/GeoFullSet.zip','r') as zf:
    with zf.open('data/VariationData/all_variations.csv','r') as f:
        geo_data = pd.read_csv(f)

In [ ]:
len(geo_data[geo_data.fail == 0])

In [ ]:
max(data['original_index'])

In [ ]:
def get_precision(pred_matches,gt_matches):
    v_exact = set((a.item(),b.item()) for a,b in pred_matches.T) 
    v_gt = set((a.item(),b.item()) for a,b in gt_matches.T)
    if len(v_exact) > 0:
        v_precision = (len(v_exact) - len(v_exact - v_gt) )/ len(v_exact)
    else:
        v_precision = 1.0
    return v_precision

In [ ]:
errors = []
records = []
for i,d in tqdm(enumerate(data['preprocessed_data'])):
    
    f_precision = get_precision(d.bl_exact_faces_matches, d.faces_matches)
    e_precision = get_precision(d.bl_exact_edges_matches, d.edges_matches)
    v_precision = get_precision(d.bl_exact_vertices_matches, d.vertices_matches)

    orig_index = data['original_index'][i]
    
    records.append({
        'index':i,
        'original_index':orig_index,
        'ps_orig':geo_data.ps_orig[orig_index],
        'ps_var':geo_data.ps_var[orig_index],
        'matchFile':geo_data.matchFile[orig_index],
        'face_precision':f_precision,
        'edge_precision':e_precision,
        'vertex_precision':v_precision
    })


In [ ]:
exact_match_precision = pd.DataFrame.from_records(records)
exact_match_precision

In [ ]:
exact_match_precision[exact_match_precision.edge_precision < 1]

In [ ]:
vert_example = exact_match_precision[exact_match_precision.vertex_precision < 1].loc[128]

In [ ]:
vert_example

In [ ]:
vert_example.ps_orig

In [ ]:
ex = exact_match_precision[exact_match_precision.vertex_precision < 1].iloc[8]
with ZipFile('../data/GeoFullSet.zip','r') as zf:
    path_orig = 'data/BrepsWithReference/' + ex.ps_orig
    path_var = 'data/BrepsWithReference/' + ex.ps_var
    path_match = 'data/Matches/' + ex.matchFile

    with zf.open(path_orig, 'r') as f:
        orig_data = f.read().decode('utf-8')
    with zf.open(path_var, 'r') as f:
        var_data = f.read().decode('utf-8')
    with zf.open(path_match, 'r') as f:
        gt_match = json.load(f)
    gt_match = [(m['val1'], m['val2']) for m in gt_match.values()]
    orig_part = Part(orig_data)
    var_part = Part(var_data)
    exact_matching = match_parts(orig_data, var_data, True)

false_positives = set(exact_matching.vertex_matches) - set(gt_match)
print('False Positives:')
print(false_positives)


orig_vert_0, var_vert_0 = list(false_positives)[0]

orig_fp_verts = set(a for a,b in false_positives)
var_fp_verts = set(b for a,b in false_positives)

orig_exact_matched_verts = set(a for a,_ in exact_matching.vertex_matches)
var_exact_matched_verts = set(b for _,b in exact_matching.vertex_matches)

orig_v_id = [i for i,f in enumerate(orig_part.brep.nodes.vertices) if f.export_id == orig_vert_0][0]
var_v_id = [i for i,f in enumerate(var_part.brep.nodes.vertices) if f.export_id == var_vert_0][0]

orig_v_ids = [i for i,f in enumerate(orig_part.brep.nodes.vertices) if f.export_id in orig_exact_matched_verts]
var_v_ids = [i for i,f in enumerate(var_part.brep.nodes.vertices) if f.export_id in var_exact_matched_verts]

orig_fp_v = [i for i,f in enumerate(orig_part.brep.nodes.vertices) if f.export_id in orig_fp_verts]
var_fp_v = [i for i,f in enumerate(var_part.brep.nodes.vertices) if f.export_id in var_fp_verts]

print(geo_data.iloc[ex.original_index].link_orig)
print(geo_data.iloc[ex.original_index].link_var)

def joint_plot_verts(orig_part, var_part, verts_orig, verts_var, offset_idx = 0):

    offset_a = orig_part.mesh.V.max(axis=0)
    offset_b = var_part.mesh.V.min(axis=0)

    for i in range(3):
        if i != offset_idx:
            offset_a[i] = 0
            offset_b[i] = 0
    offset_a *= 1.05
    offset_b *= 1.05

    verts_orig = set(verts_orig)
    verts_var = set(verts_var)

    verts_orig_idx = [i for i,v in enumerate(orig_part.mesh_topology.point_to_topology) if v in verts_orig]
    verts_var_idx = [i for i,v in enumerate(var_part.mesh_topology.point_to_topology) if v in verts_var]

    vert_locs = np.concatenate([orig_part.mesh.V[verts_orig_idx] + offset_a, var_part.mesh.V[verts_var_idx] + offset_b],axis=0)

    return mp.plot(
        np.concatenate(
            [
                orig_part.mesh.V + offset_a, 
                var_part.mesh.V + offset_b
            ]),
        np.concatenate(
            [
                orig_part.mesh.F, 
                (var_part.mesh.F + orig_part.mesh.V.shape[0])
            ]),
        return_plot=True
    ).add_points(
        vert_locs,
        shading={'point_size':.005}
    )
joint_plot_verts(orig_part, var_part, orig_fp_v, var_fp_v , 1)

In [ ]:
zf = ZipFile('../data/GeoFullSet.zip','r')

In [ ]:
path_match == 'data/Matches/6e4ad64270ef3a3c41addd31_45614c7621587c07bd2ab7fa_9072204cb5aa216204767a09_default_jjdeiV02.json'

In [ ]:
zf.close()

In [ ]:
ex.to_csv('bad_bolts.csv')

In [ ]:
(exact_match_precision.face_precision < 1).sum()

In [ ]:
ex = exact_match_precision[exact_match_precision.face_precision < 1].iloc[15]
with ZipFile('../data/GeoFullSet.zip','r') as zf:
    path_orig = 'data/BrepsWithReference/' + ex.ps_orig
    path_var = 'data/BrepsWithReference/' + ex.ps_var
    path_match = 'data/Matches/' + ex.matchFile

    with zf.open(path_orig, 'r') as f:
        orig_data = f.read().decode('utf-8')
    with zf.open(path_var, 'r') as f:
        var_data = f.read().decode('utf-8')
    with zf.open(path_match, 'r') as f:
        gt_match = json.load(f)
    gt_match = [(m['val1'], m['val2']) for m in gt_match.values()]
    orig_part = Part(orig_data)
    var_part = Part(var_data)
    exact_matching = match_parts(orig_data, var_data, True)

false_positives = set(exact_matching.face_matches) - set(gt_match)
print('False Positives:')
print(false_positives)
orig_face0, var_face0 = list(false_positives)[0]
orig_exact_matched_faces = set(a for a,_ in exact_matching.face_matches)
var_exact_matched_faces = set(b for _,b in exact_matching.face_matches)
orig_f_id = [i for i,f in enumerate(orig_part.brep.nodes.faces) if f.export_id == orig_face0][0]
var_f_id = [i for i,f in enumerate(var_part.brep.nodes.faces) if f.export_id == var_face0][0]
orig_f_ids = [i for i,f in enumerate(orig_part.brep.nodes.faces) if f.export_id in orig_exact_matched_faces]
var_f_ids = [i for i,f in enumerate(var_part.brep.nodes.faces) if f.export_id in var_exact_matched_faces]
face_colors = np.concatenate([(orig_part.mesh_topology.face_to_topology == orig_f_id).astype(int),
(var_part.mesh_topology.face_to_topology == var_f_id).astype(int)
]).astype(float)

all_face_colors = np.concatenate(
[
np.array([1.0 if i in orig_f_ids else 0.0 for i in orig_part.mesh_topology.face_to_topology]),
np.array([1.0 if i in var_f_ids else 0.0 for i in var_part.mesh_topology.face_to_topology])
]
)
print(geo_data.iloc[ex.original_index].link_orig)
print(geo_data.iloc[ex.original_index].link_var)
def joint_plot(orig_part, var_part, face_colors, offset_idx = 0):

    offset_a = orig_part.mesh.V.max(axis=0)
    offset_b = var_part.mesh.V.min(axis=0)

    for i in range(3):
        if i != offset_idx:
            offset_a[i] = 0
            offset_a[i] = 0
    offset_a *= 1.05
    offset_b *= 1.05

    return mp.plot(
        np.concatenate(
            [
                orig_part.mesh.V + offset_a, 
                var_part.mesh.V + offset_b
            ]),
        np.concatenate(
            [
                orig_part.mesh.F, 
                (var_part.mesh.F + orig_part.mesh.V.shape[0])
            ]),
        c = face_colors
    )
joint_plot(orig_part, var_part, face_colors)

In [ ]:
def joint_plot(orig_part, var_part, face_colors):
    return mp.plot(
        np.concatenate(
            [
                orig_part.mesh.V, 
                (var_part.mesh.V)
            ]),
        np.concatenate(
            [
                orig_part.mesh.F, 
                (var_part.mesh.F + orig_part.mesh.V.shape[0])
            ]),
        c = np.concatenate([np.zeros(orig_part.mesh.F.shape[0]), np.ones(var_part.mesh.F.shape[0])]).astype(float)
    )

In [ ]:
face_colors = np.concatenate([(orig_part.mesh_topology.face_to_topology == orig_f_id).astype(int),
(var_part.mesh_topology.face_to_topology == var_f_id).astype(int)
]).astype(float)

In [ ]:
len(exact_matching.face_matches)

In [ ]:
orig_faces = set(f.export_id for f in orig_part.brep.nodes.faces)

In [ ]:
[(a,b) for a,b in gt_match if a in orig_faces]

In [ ]:
len(orig_part.mesh.F) + len(var_part.mesh.F)

In [ ]:
len(all_face_colors)

In [ ]:
zf = ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r')

In [ ]:
[n for n in zf.namelist() if n.endswith('.csv')]

In [ ]:
with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
    variations = pd.read_csv(f)

In [ ]:
data = variations[:100]
data = data[data.fail == 0]

In [ ]:
incomplete = []
incompatible = []
diff_geo = []

m_f = []
m_e = []
m_v = []

A = []
B = []
M = []

for i in tqdm(range(len(data))):
    d = data.iloc[i]
    path_a = 'data/BrepsWithReference/' + d.ps_orig
    path_b = 'data/baseline/' + d.baselineOrig
    
    with zf.open(path_a,'r') as f:
        part_data_a = f.read().decode('utf-8')
    with zf.open(path_b,'r') as f:
        part_data_b = f.read().decode('utf-8')

    p_a = Part(part_data_a)
    p_b = Part(part_data_b)

    parts_have_same_num = len(p_a.brep.nodes.faces) == len(p_b.brep.nodes.faces) and len(p_a.brep.nodes.edges) == len(p_b.brep.nodes.edges) and len(p_a.brep.nodes.vertices) == len(p_b.brep.nodes.vertices)

    matching = match_parts(part_data_a, part_data_b, True)
    complete_matching = len(matching.face_matches) == len(p_a.brep.nodes.faces) and len(matching.edge_matches) == len(p_a.brep.nodes.edges) and len(matching.vertex_matches) == len(p_a.brep.nodes.vertices)

    n_face_matches_missing = -len(matching.face_matches) + len(p_a.brep.nodes.faces)
    n_edge_matches_missing = -len(matching.edge_matches) + len(p_a.brep.nodes.edges)
    n_vertex_matches_missing =-len(matching.vertex_matches) + len(p_a.brep.nodes.vertices)

    m_f.append(n_face_matches_missing)
    m_e.append(n_edge_matches_missing)
    m_v.append(n_vertex_matches_missing)

    exact_geo = (p_a.mesh.V == p_b.mesh.V).all()

    if not parts_have_same_num:
        incompatible.append(i)
    if not complete_matching:
        incomplete.append(i)

    if not exact_geo:
        diff_geo.append(i)

    A.append(p_a)
    B.append(p_b)
    M.append(matching)

In [ ]:
zf.close()

In [ ]:
incomplete

In [ ]:
[(i,e) for i,e in enumerate(m_e) if e < 0]

In [ ]:
k = 24
p_a = A[k]
p_b = B[k]
matching = M[k]

In [ ]:
e1 = [(id,count) for id,count in Counter([m[0] for m in matching.edge_matches]).items() if count > 1][0][0]
e2 = [b for (a,b) in matching.edge_matches if a == e1 and b != e1][0]
e1_i = [i for i,e in enumerate(p_a.brep.nodes.edges) if e.export_id == e1][0]
e2_i = [i for i,e in enumerate(p_a.brep.nodes.edges) if e.export_id == e2][0]

In [ ]:
E = []
for i in range(p_a.mesh_topology.edge_to_topology.shape[0]):
    for j in range(p_a.mesh_topology.edge_to_topology.shape[1]):
        if p_a.mesh_topology.edge_to_topology[i,j] == e1_i:
            E.append([p_a.mesh.F[i,(j-1)%3],p_a.mesh.F[i,j]])
E = np.array(E)
mp.plot(p_a.mesh.V, p_a.mesh.F,return_plot=True).add_edges(p_a.mesh.V, E)

In [ ]:
E = []
for i in range(p_a.mesh_topology.edge_to_topology.shape[0]):
    for j in range(p_a.mesh_topology.edge_to_topology.shape[1]):
        if p_a.mesh_topology.edge_to_topology[i,j] == e2_i:
            E.append([p_a.mesh.F[i,(j-1)%3],p_a.mesh.F[i,j]])
E = np.array(E)
mp.plot(p_a.mesh.V, p_a.mesh.F,return_plot=True).add_edges(p_a.mesh.V, E)

In [ ]:
E27

In [ ]:
E27 = []
for t,k in edges_27:
    v1 = p_a.mesh.F[i][(k-1)%3]
    v2 = p_a.mesh.F[i][(k)]
    E27.append([v1,v2])
E27 = np.array(E27)


In [ ]:
M[21].edge_matches

In [ ]:
min(m_e)

In [ ]:
np.max(A[23].mesh.V - B[23].mesh.V)

In [ ]:
len(part_data_a)

In [ ]:
len(M[21].vertex_matches)

In [ ]:
len(A[21].brep.nodes.vertices)

In [ ]:
def highlight_edge_examples(ex, offset_dir=0):
    with ZipFile('../data/GeoFullSet.zip','r') as zf:
        path_orig = 'data/BrepsWithReference/' + ex.ps_orig
        path_var = 'data/BrepsWithReference/' + ex.ps_var
        path_match = 'data/Matches/' + ex.matchFile

        with zf.open(path_orig, 'r') as f:
            orig_data = f.read().decode('utf-8')
        with zf.open(path_var, 'r') as f:
            var_data = f.read().decode('utf-8')
        with zf.open(path_match, 'r') as f:
            gt_match = json.load(f)
        gt_match = [(m['val1'], m['val2']) for m in gt_match.values()]
        orig_part = Part(orig_data)
        var_part = Part(var_data)
        exact_matching = match_parts(orig_data, var_data, True)

    false_positives = set(exact_matching.edge_matches) - set(gt_match)
    print('False Positives:')
    print(false_positives)


    #orig_edge_0, var_edge_0 = list(false_positives)[0]

    orig_fp_edges = set(a for a,b in false_positives)
    var_fp_edges = set(b for a,b in false_positives)

    orig_exact_matched_edges = set(a for a,_ in exact_matching.edge_matches)
    var_exact_matched_edges = set(b for _,b in exact_matching.edge_matches)

    #orig_e_id = [i for i,f in enumerate(orig_part.brep.nodes.edges) if f.export_id == orig_vert_0][0]
    #var_e_id = [i for i,f in enumerate(var_part.brep.nodes.edges) if f.export_id == var_vert_0][0]

    orig_e_ids = [i for i,f in enumerate(orig_part.brep.nodes.edges) if f.export_id in orig_fp_edges]
    var_e_ids = [i for i,f in enumerate(var_part.brep.nodes.edges) if f.export_id in var_fp_edges]



    #orig_fp_e = [i for i,f in enumerate(orig_part.brep.nodes.edges) if f.export_id in orig_fp_verts]
    #var_fp_e = [i for i,f in enumerate(var_part.brep.nodes.edges) if f.export_id in var_fp_verts]

    print(geo_data.iloc[ex.original_index].link_orig)
    print(geo_data.iloc[ex.original_index].link_var)

    def joint_plot_edges(orig_part, var_part, e_ids_orig, e_ids_var, offset_idx = 0):

        offset_a = orig_part.mesh.V.max(axis=0)
        offset_b = var_part.mesh.V.min(axis=0)

        for i in range(3):
            if i != offset_idx:
                offset_a[i] = 0
                offset_b[i] = 0
        offset_a *= 1.05
        offset_b *= 1.05

        e_ids_orig = set(e_ids_orig)
        e_ids_var = set(e_ids_var)

        E1 = []
        for i in range(orig_part.mesh_topology.edge_to_topology.shape[0]):
            for j in range(orig_part.mesh_topology.edge_to_topology.shape[1]):
                if orig_part.mesh_topology.edge_to_topology[i,j] in e_ids_orig:
                    E1.append([orig_part.mesh.F[i,(j-1)%3],orig_part.mesh.F[i,j]])
        E2 = []
        for i in range(var_part.mesh_topology.edge_to_topology.shape[0]):
            for j in range(var_part.mesh_topology.edge_to_topology.shape[1]):
                if var_part.mesh_topology.edge_to_topology[i,j] in e_ids_var:
                    E2.append([var_part.mesh.F[i,(j-1)%3],var_part.mesh.F[i,j]])

        E1 = np.array(E1)
        E2 = np.array(E2)

        E = np.concatenate([E1, E2+orig_part.mesh.V.shape[0]],axis=0)

        V = np.concatenate(
                [
                    orig_part.mesh.V + offset_a,
                    var_part.mesh.V + offset_b
                ])
        F = np.concatenate(
                [
                    orig_part.mesh.F,
                    (var_part.mesh.F + orig_part.mesh.V.shape[0])
                ])
        return mp.plot(
            V, F, return_plot=True
        ).add_edges(V, E, shading={'point_size':.005})

    joint_plot_edges(orig_part, var_part, orig_e_ids, var_e_ids, offset_dir)

In [ ]:
ex = exact_match_precision[exact_match_precision.edge_precision < 1].iloc[17]
highlight_edge_examples(ex, 1)

In [ ]:
def compute_edge_precision(ex):
    with ZipFile('../data/GeoFullSet.zip','r') as zf:
        path_orig = 'data/BrepsWithReference/' + ex.ps_orig
        path_var = 'data/BrepsWithReference/' + ex.ps_var
        path_match = 'data/Matches/' + ex.matchFile

        with zf.open(path_orig, 'r') as f:
            orig_data = f.read().decode('utf-8')
        with zf.open(path_var, 'r') as f:
            var_data = f.read().decode('utf-8')
        with zf.open(path_match, 'r') as f:
            gt_match = json.load(f)
        gt_match = [(m['val1'], m['val2']) for m in gt_match.values()]
        orig_part = Part(orig_data)
        var_part = Part(var_data)
        exact_matching = match_parts(orig_data, var_data, True)

    false_positives = set(exact_matching.edge_matches) - set(gt_match)
    return (len(exact_matching.edge_matches) - len(false_positives)) / len(exact_matching.edge_matches)

redone_edge_precisions = []
edge_fails = exact_match_precision[exact_match_precision.edge_precision < 1]
for i in tqdm(range(len(edge_fails))):
    ex = edge_fails.iloc[i]
    redone_edge_precisions.append(compute_edge_precision(ex))

In [ ]:
sum(np.array(redone_edge_precisions) == 1) / len(redone_edge_precisions)

In [ ]:
(np.array(redone_edge_precisions) - edge_fails.edge_precision > 0).sum()